https://code-angie.tistory.com/49

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
!pip install olefile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35417 sha256=2f84cb37b962c0ec4b2fe480385eea6a609def721e2d02acd90db8aec0be56a6
  Stored in directory: /root/.cache/pip/wheels/02/39/c0/9eb1f7a42b4b38f6f333b6314d4ed11c46f12a0f7b78194f0d
Successfully built olefile


In [3]:
import olefile
import zlib
import struct

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")

        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()

    # 파일 불러오기
    def load(self, filename):
        return olefile.OleFileIO(filename)

    # hwp 파일인지 확인 header가 없으면 hwp가 아닌 것으로 판단하여 진행 안함
    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

	# 문서 포맷 압축 여부를 확인
    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

	# bodytext의 section들 목록을 저장
    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]

    # text를 뽑아내는 함수
    def get_text(self):
        return self.text

	# 전체 text 추출
    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

	# section 내 text 추출
    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self.is_compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                text += rec_data.decode('utf-16')
                text += "\n"

            i += 4 + rec_len

        return text

# text 추출 함수 -> 이 함수를 사용하면 됨
def get_text(filename):
    hwp = HWPExtractor(filename)
    return hwp.get_text()

In [4]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.3 MB/s eta 0:00:00


In [5]:
import re
from os import listdir
from os.path import isfile, join
import os
import pandas as pd
from datetime import datetime, date

In [6]:
path = "/content/drive/Shareddrives/4조/data/minutes/"
savepath='/content/drive/Shareddrives/4조/data/minutes_result/'
hwp_files = [f for f in listdir(path) if isfile(join(path, f))]
# hwp_files

In [9]:
def makeone_hwp():
      # 지정 폴더 내 파일 목록 조회 (파일만)
    hwp_files = [f for f in listdir(path) if isfile(join(path, f))]

    result = []
    fail=[]
    for hwp in hwp_files :
        time = hwp.split('(')
        time = time[2][:-5]

        hwp_filepath = path + hwp
        # hwp 파일을 text로 변환
        try:
          hwptext = get_text(hwp_filepath)
          hwptext = hwptext.split("6. 회의경과")[1]
          hwptext = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", hwptext)
          new_dict = {
              'time' : time,
              'text' : hwptext
          }
          result.append(new_dict)
          print(hwp, ' 완료')
        except:
          print(hwp_filepath, "실패")
          fail.append(hwp_filepath)


    df = pd.DataFrame(result)
    save_name = "BOK의사록2.xlsx"
    txt_name = os.path.join(savepath, save_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')

    df.to_excel(excel_writer)

    excel_writer.save()
    return fail


In [10]:
fail_list = makeone_hwp()

금융통화위원회 의사록(2023년도 제14차)(2023.7.27).pdf  완료
금융통화위원회 의사록(2010년도 제2차)(2010.1.8).hwp  완료
금융통화위원회 의사록(2009년도 제27차)(2009.12.24).hwp  완료
금융통화위원회 의사록(2009년도 제25차)(2009.11.26).hwp  완료
금융통화위원회 의사록(2009년도 제26차)(2009.12.10).hwp  완료
금융통화위원회 의사록(2009년도 제4차)(2009.2.12).hwp  완료
금융통화위원회 의사록(2009년도 제3차)(2009.1.22).hwp  완료
금융통화위원회 의사록(2009년도 제24차)(2009.11.12).hwp  완료
금융통화위원회 의사록(2009년도 제22차)(2009.10.9).hwp  완료
금융통화위원회 의사록(2009년도 제2차)(2009.1.9).hwp  완료
금융통화위원회 의사록(2009년도 제21차)(2009.9.24).hwp  완료
금융통화위원회 의사록(2009년도 제1차)(2009.1.2).hwp  완료
금융통화위원회 의사록(2008년도 제27차)(2008.12.24).hwp  완료
금융통화위원회 의사록(2008년도 제26차)(2008.12.11).hwp  완료
금융통화위원회 의사록(2008년도 제25차)(2008.12.3).hwp  완료
금융통화위원회 의사록(2009년도 제18ᄎ

In [7]:
hwp_files[275:]

['금융통화위원회 의사록(2023년도 제6차)(2023.3.23).hwp',
 '금융통화위원회 의사록(2023년도 제4차)(2023.2.23).hwp',
 '금융통화위원회 의사록(2023년도 제1차)(2023.1.13).hwp',
 '금융통화위원회 의사록(2022년도 제24차)(2022.12.22).hwp',
 '금융통화위원회 의사록(2022년도 제23차)(2022.12.8).hwp',
 '금융통화위원회 의사록(2022년도 제22차)(2022.11.24).hwp',
 '금융통화위원회 의사록(2022년도 제21차)(2022.11.10).hwp',
 '금융통화위원회 의사록(2022년도 제20차)(2022.10.27).hwp',
 '금융통화위원회 의사록(2022년도 제19차)(2022.10.12).hwp',
 '금융통화위원회 의사록(2022년도 제18차)(2022.9.22).hwp',
 '금융통화위원회 의사록(2022년도 제17차)(2022.9.8) .hwp',
 '금융통화위원회 의사록(2022년도 제16차)(2022.8.25) .hwp',
 '금융통화위원회 의사록(2022년도 제13차)(2022.7.13).hwp',
 '금융통화위원회 의사록(2022년도 제14차)(2022.7.28) .hwp',
 '금융통화위원회 의사록(2022년도 제12차)(2022.6.22) .hwp',
 '금융통화위원회 의사록(2022녀

In [11]:
fail_list

['/content/drive/Shareddrives/4조/data/minutes/금융통화위원회 의사록(2018년도 제13차)(2018.7.12.) .hwp',
 '/content/drive/Shareddrives/4조/data/minutes/금융통화위원회 의사록(2018년도 제10차)(2018.5.24.) .hwp',
 '/content/drive/Shareddrives/4조/data/minutes/금융통화위원회 의사록(2018년도 제7차)(2018.4.12.).hwp']

In [12]:
# 의사록 time 컬럼 datetime으로 변환
minutes = pd.read_excel("/content/drive/Shareddrives/4조/data/minutes_result/BOK의사록2.xlsx", usecols=[1, 2])
minutes['time'] = minutes['time'].astype(str)

date_list = []
for time in minutes['time']:
    date_list.extend(re.findall('[0-9]{4}.[0-9]{1,2}.[0-9]{1,2}', time))
minutes['time'] = date_list
minutes['time'] = pd.to_datetime(minutes['time'], format='%Y.%m.%d')

minutes

,time,text
0,2023-07-27,_x000D_\n가 의결안건_x000D_\n의안 제31호 한국은행의 금융기관대출규...
1,2010-01-08,_x000D_\n 가 의결안건_x000D_\n의안 제2호 통화정책방향_x000D_...
2,2009-12-24,_x000D_\n가 의결안건_x000D_\n의안 제60호 2010년 14분기 한국...
3,2009-11-26,_x000D_\n가 의결안건_x000D_\n의안 제54호 2010년 이후 중기 물...
4,2009-12-10,_x000D_\n 가 의결안건_x000D_\n의안 제56호 통화정책방향_x000D...
...,...,...
293,2023-07-27,_x000D_\n가 의결안건_x000D_\n의안 제31호 한국은행의 금융기관대출규...
294,2023-06-08,_x000D_\n가 의결안건_x000D_\n의안 제25호 통화신용정책보고서2023...
295,2023-06-21,_x000D_\n가 의결안건_x000D_\n의안 제28호 금융안정보고서2023년 ...
296,2023-05-25,_x000D_\n의안 제24호 통화정책방향_x000D_\n_x001F_1_x001...


In [14]:
# 연도가 2009~2021년인 의사록 filtering
minutes = minutes[(minutes['time'].dt.year >= 2009) & (minutes['time'].dt.year <= 2021)]
minutes

,time,text
33,2021-12-23,_x000D_\n가 의결안건_x000D_\n의안 제38호 금융안정보고서2021년 ...
34,2021-12-09,_x000D_\n가 의결안건_x000D_\n의안 제37호 통화신용정책보고서2021...
35,2021-11-25,_x000D_\n 가 의결안건_x000D_\n의안 제36호 통화정책방향_x000D...
36,2021-10-12,_x000D_\n 가 의결안건_x000D_\n의안 제32호 통화정책방향_x000D...
38,2021-09-24,_x000D_\n가 보고안건_x000D_\n보고 제59호 금융안정 상황점검_x00...
...,...,...
31,2009-02-25,_x000D_\n 가 의결안건_x000D_\n의안 제12호 금융기관 자본확충 지원...
5,2009-02-12,_x000D_\n 가 의결안건_x000D_\n의안 제10호 통화정책방향_x000D...
6,2009-01-22,_x000D_\n가 의결안건_x000D_\n의안 제5호 한국은행의 금융기관대출규정...
9,2009-01-09,_x000D_\n 가 의결안건_x000D_\n의안 제2호 통화정책방향_x000D_...


In [15]:
# 텍스트 전처리 함수
def text_filtering(text):
    text = re.sub('\\n{2,}','', text)
    text = re.sub('- \d+ -', '', text)
    text = re.sub('―|｢|｣|-|[․/→←+]', '', text)
    text = re.sub('^.*hwp*\w*[A-Za-z]', '', text)
    return text

minutes['text'] = minutes['text'].apply(text_filtering)
minutes

,time,text
33,2021-12-23,_x000D_\n가 의결안건_x000D_\n의안 제38호 금융안정보고서2021년 ...
34,2021-12-09,_x000D_\n가 의결안건_x000D_\n의안 제37호 통화신용정책보고서2021...
35,2021-11-25,_x000D_\n 가 의결안건_x000D_\n의안 제36호 통화정책방향_x000D...
36,2021-10-12,_x000D_\n 가 의결안건_x000D_\n의안 제32호 통화정책방향_x000D...
38,2021-09-24,_x000D_\n가 보고안건_x000D_\n보고 제59호 금융안정 상황점검_x00...
...,...,...
31,2009-02-25,_x000D_\n 가 의결안건_x000D_\n의안 제12호 금융기관 자본확충 지원...
5,2009-02-12,_x000D_\n 가 의결안건_x000D_\n의안 제10호 통화정책방향_x000D...
6,2009-01-22,_x000D_\n가 의결안건_x000D_\n의안 제5호 한국은행의 금융기관대출규정...
9,2009-01-09,_x000D_\n 가 의결안건_x000D_\n의안 제2호 통화정책방향_x000D_...


In [13]:
minutes = minutes.sort_values(by='time' ,ascending=False)
minutes

,time,text
0,2023-07-27,_x000D_\n가 의결안건_x000D_\n의안 제31호 한국은행의 금융기관대출규...
293,2023-07-27,_x000D_\n가 의결안건_x000D_\n의안 제31호 한국은행의 금융기관대출규...
297,2023-07-13,_x000D_\n의안 제30호 통화정책방향_x000D_\n_x001F_1_x001...
295,2023-06-21,_x000D_\n가 의결안건_x000D_\n의안 제28호 금융안정보고서2023년 ...
294,2023-06-08,_x000D_\n가 의결안건_x000D_\n의안 제25호 통화신용정책보고서2023...
...,...,...
13,2008-12-11,_x000D_\n 가 의결안건_x000D_\n의안 제54호 통화정책방향_x000D...
14,2008-12-03,_x000D_\n 가 의결안건_x000D_\n의안 제52호 공개시장조작규정 개정안...
18,2008-11-07,_x000D_\n 가 의결안건_x000D_\n의안 제49호 통화정책방향_x000D...
22,2008-10-27,_x000D_\n 가 의결안건_x000D_\n의안 제46호 통화정책방향_x000D...


In [16]:
minutes.to_excel('/content/drive/Shareddrives/4조/data/minutes_result/BOK의사록(전처리완료).xlsx')

### 테스트

In [ ]:
path = "/content/drive/Shareddrives/4조/data/제12차 금통위 의사록.hwp"
#path = "/content/drive/Shareddrives/4조/data/제14차 금통위 의사록.hwp"
data = get_text(path)

In [ ]:
data = data.split("6. 회의경과")[1]

In [ ]:
import re
new_str = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", data)
new_str.replace("", "")
print(new_str)


 가 의결안건
의안 제30호  통화정책방향
 전일 개최된 동향 보고회의에서 조사국장이 보고 제49호  국내외 경제동향 국제국장이 보고 제50호  외환국제금융 동향 그리고 금융시장국장이 보고 제51호  금융시장 동향에 대하여 보고하였음보고내용  별첨
 본회의에서는 의장이 한국은행법제28조에 의거하여 의안 제30호  통화정책방향을 상정하였음
 위원 토의내용
국내외 경제동향과 관련하여 일부 위원은 최근 소비자심리지수CSI 및 기업경기실사지수BSI 등이 큰 폭으로 상승하는 등 낙관적인 기대가 빠르게 확산되고 있는 현상을 어떻게 평가하는지 물어보았으며 이에 대해 관련부서에서는 최근 민간의 심리지표 개선은 주가 상승 등 금융시장 안정에 주로 기인하는 것으로 보인다고 답변하였음
다른 일부 위원은 상반기중 재정지출의 대규모 조기집행이 하반기 경기흐름에 미치는 영향에 대해 물어보았으며 이에 대해 관련부서에서는 재정집행이 상반기에 집중되더라도 경제활동으로 이어지는 시차 및 추경예산 집행 등을 감안할 때 재정부문의 경기부양효과가 하반기에 축소될 것으로 생각하지는 않는다고 답변하였음
또한 동 위원은 시중유동성이 설비투자 등 실물경제활동에 효과적으로 이어지도록 하는 방안에 대해 물어보았으며 이에 대해 관련부서에서는 기업구조조정 등과 관련한 불확실성의 해소가 선결되어야 한다고 답변하였음 이와 관련하여 또 다른 일부 위원은 실물부문으로 자금이 공급되지 않는 원인이 몇달전의 극심한 신용경색기와는 달리 향후 경기 불투명 등으로 기업들의 자금수요가 감소한 영향도 일부 있다는 의견을 개진하였음
일부 위원은 선진국 경제의 회복 없이 BRICs 등 신흥시장국 경기가 먼저 회복되는 경우 우리나라 수출에 미치는 영향에 대해 물어보았으며 이에 대해 관련부서에서는 신흥시장국의 경기가 회복되더라도 정부의 내수부양정책에서 비롯될 가능성이 크다는 점 등을 감안할 경우 과거 선진국 수출을 통해서 신흥시장국이 성장할 때와는 달리 우리 수출에 긍정적으로 미치는 영향은 다소 제한적이 될 것이라고 답변하였음
다른 일